# Utils

In [1]:
import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt


imgCol = cv2.imread('../Utils/LenaJPEG.jpg')
width, height, depth = imgCol.shape

def plotSignal(signal=[], title='', xmin = 0, xmax = 0):
  if(xmax == 0):
    xmax = signal.size

  plt.title(title)
  plt.plot(signal)
  plt.xlim(xmin, xmax)
  plt.show() 

def show_Image(img):
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAll

# Praktikum 01

In [3]:
def printNumberFrom1To100():
    for i in range(1, 101):
        print(i)

def fibunacci(n):
    a = [0]*(n+4)
    a[0] = 1
    a[1] = 1
    print(0)
    print(1)
    print(1)
    for i in range(2,n+3):
        a[i] = a[i-1]+a[i-2]
        print(a[i])

# fibunacci(10)

# for y in range(height):
#     for x in range(width):
#         imgCol[y][x] = pow(imgCol[y][x]/255, 1/2) * 255



# cv2.imshow("img", imgCol)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


# Praktikum 02

In [2]:
def getHistogramm(img=[]):
    histogram=np.zeros(256)
    
    for y in range(height):
        for x in range(width):
            if img.ndim == 3:
                i = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            else:
                i = img[y][x]

            histogram[int(i)] += 1

    return histogram

def getNormHistogramm(img=[]):
    return getHistogramm(img) / (width*height)

def getCumulatedHistogramm(img):
    H = np.zeros(256)
    h = getNormHistogramm(img)
    for i in range(len(h)):
        for j in range(i): 
            H[i] += h[j]
    return H

def linearContrast(img=[], t0=0, t1=255):
    toRet = np.zeros((height, width), dtype='uint8')
    for y in range(height):
        for x in range(width):
            a = float(0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2])
            if a < t0:
                toRet[y][x] = 0
            if t0 <= a < t1:
                toRet[y][x] = 255/(t1-t0) * (a-t0)
            if a >= t1:
                toRet[y][x] = 255
    return toRet

def autoContrast(img, schwellwert):
    a1_low = getPercentage(img, schwellwert)
    a1_high = getPercentage(img, 1-schwellwert)
    return linearContrast(img, a1_low, a1_high)

def getPercentage(img, schwellwert):
    H = getCumulatedHistogramm(img)
    toRet = 255
    for i in range(len(H)):
        if H[i] >= schwellwert:
            toRet = i
            break
    return toRet

def getMedianOfHistogramm(img):
    H = getCumulatedHistogramm(img)
    for i in range(len(H)):
        if H[i] >= 0.5:
            return getHistogramm(img)[i]


def getBinary(img=[], schwellwert=125):
    toRet = np.zeros((height, width), dtype='uint8')
    for y in range(height):
        for x in range(width):
            a = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            if a < schwellwert:
                toRet[y][x] = 0
            else:
                toRet[y][x] = 1
    return toRet

def same_image(img=[]):
    toRet = np.zeros((height, width), dtype='uint8')
    for y in range(height):
        for x in range(width):
            # a = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            a = img[y][x][0]
            toRet[y][x] = int(a)
            
    return toRet

testing Praktikum 02

In [5]:
# print(getMedianOfHistogramm(imgCol))


auto_con = autoContrast(imgCol, 0.1)

# plotSignal(getCumulatedHistogramm(imgCol))
show_Image(auto_con)
# show_Image(imgCol)

AttributeError: module 'cv2' has no attribute 'destroyAll'